In [ ]:

import numpy as np
import pandas


import skimage.data
import skimage.transform
from skimage import data, io, filters

In [ ]:
import math
import os

width = 224
height = 168
# Import Labels
labels = []
imagePaths = []
lastTrackingAngle = 0
lastSpeed = 0
for i in range(20,41):
    fname = "/home/matt/Desktop/RainierDrive/AccelerationData/capture"+str(i)+"/accelClass.txt"
    base = "/home/matt/Desktop/RainierDrive/AccelerationData/capture"+str(i) + "/"
    
    if(not os.path.isfile(fname)):
        print("Could not find file for capture ", str(i))
        continue
    with open(fname) as f:
        content = f.readlines()
        for line in content:
            line = line.split(",")
            speedString = (line[1].split(" "))[2]
            speed = float(speedString)
            trackingAngleString = (line[8].split(" "))[2]
            trackingAngle = float(trackingAngleString)
            deltaTrackingAngle = trackingAngle - lastTrackingAngle
            deltaSpeed = speed - lastSpeed
            lastTrackingAngle = trackingAngle
            lastSpeed = speed
            filePath = (line[10])
            if speed > 0 and abs(deltaTrackingAngle) > 0 and abs(deltaTrackingAngle) < 7:
                if(deltaTrackingAngle>0):
                    deltaTrackingAngle = 1
                else:
                    deltaTrackingAngle = -1
                speed = speed/10
                speed = int(speed)
                speedArray = np.zeros((8))
                speedArray[speed] = 1
                labels.append(speedArray)
                #labels.append(deltaTrackingAngle)
                #print speed
                #print trackingAngle
                picturePath = base+filePath
                #print picturePath
                imagePaths.append(picturePath)

trainingIndex = int(len(imagePaths)/1.4)
print trainingIndex

In [ ]:
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters

from skimage.color import rgb2gray
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters
import operator
from operator import sub


@adapt_rgb(each_channel)
def sobel_each(image):
    return filters.sobel(image)

def as_gray(image_filter, image, *args, **kwargs):
    gray_image = rgb2gray(image)
    return image_filter(gray_image, *args, **kwargs)

@adapt_rgb(as_gray)
def sobel_gray(image):
    return filters.sobel(image)



images = np.zeros((len(imagePaths),height,width,3))
print(images.shape)
threadCount = 20
import threading

def worker(loadPath, loadPath2, imageIndex):
    """thread worker function"""
    #image = skimage.data.imread(loadPath)
    image2 = skimage.data.imread(loadPath2)
    #image = skimage.transform.resize(image, (height, width), mode='reflect')
    image2 = skimage.transform.resize(image2, (height, width), mode='reflect')
    #edges = sobel_each(image)
    #edges2 = sobel_each(image2)
    #edges3 = edges2+edges
    images[imageIndex] = image2
    return imageIndex

for imNum in range(0,len(imagePaths),threadCount):
    threads = []
    for i in range(threadCount):
        
        if(imNum+i >= len(imagePaths)):
            continue
        if(imNum+1==0):
            loadPath = imagePaths[mNum+i]
            loadPath2 = imagePaths[imNum+i]
        else:
            loadPath = imagePaths[imNum+i-1]
            loadPath2 = imagePaths[imNum+i]
        t = threading.Thread(target=worker, args=(loadPath,loadPath2,imNum+i))
        threads.append(t)
        t.start()

In [ ]:

image = images[23]
print(images[23])
io.imshow(image)
io.show()

In [ ]:

images = np.asarray(images)
images = images.reshape(len(images),1,width*height*3)

In [ ]:
labels = np.array(labels)
images = np.array(images)
print("labels: ", labels.shape, "\nimages: ", images.shape)

In [ ]:
maxLength = trainingIndex
global currentPos
currentPos = 0
def next(batch_size):
    """ Return a batch of data. When dataset end is reached, start over.
    """
    #if self.batch_id == len(self.data):
    #    self.batch_id = 0
    #batch_data = (self.data[self.batch_id:min(self.batch_id +
    #                                          batch_size, len(self.data))])
    #batch_labels = (self.labels[self.batch_id:min(self.batch_id +
    #                                          batch_size, len(self.data))])
    #batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
    #                                          batch_size, len(self.data))])
    #self.batch_id = min(self.batch_id + batch_size, len(self.data))
    global currentPos
    currentPos = currentPos + batch_size
    maxPos = currentPos + batch_size
    if(maxPos>maxLength):
        currentPos = 0
        maxPos = currentPos + batch_size
    batch_data = (images[currentPos:maxPos])
    batch_labels = (labels[currentPos:maxPos])
    return batch_data, batch_labels



In [ ]:
maxTestingLength = len(labels)
global currentTestingPos
global minIndex 
minIndex = trainingIndex
currentTestingPos = minIndex

def nextTesting(batch_size):
    global currentTestingPos
    global minIndex
    currentTestingPos = currentTestingPos + batch_size
    maxPos = currentTestingPos + batch_size
    if(maxPos>maxTestingLength):
        currentTestingPos = minIndex
        maxPos = currentTestingPos + batch_size
    batch_data = (images[currentTestingPos:maxPos])
    batch_labels = (labels[currentTestingPos:maxPos])
    return batch_data, batch_labels

In [ ]:
x, y = next(128)
print(x.shape)
print(y)

x, y = nextTesting(128)
print(x.shape)
print(y)

cellInitialized = 0

In [ ]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 64
display_step = 10

# Network Parameters
n_hidden_1 = 1280 # 1st layer number of features
n_hidden_2 = 640 # 2nd layer number of features
n_input = width*height*3 # MNIST data input (img shape: 28*28)
n_classes = 8 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))

# Calculate accuracy
modelaccuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))


# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
sess=tf.Session()
sess.run(init)



In [ ]:
training_iters = 90000
display_step = 10
step = 1
# Keep training until reach max iterations
while step * batch_size < training_iters:
    batch_x, batch_y = next(batch_size)
    # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, n_input))
    batch_y = batch_y.reshape((batch_size, 8))
    # Run optimization op (backprop)
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
    if step % display_step == 0:
        # Calculate batch accuracy
        mc = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
        acc = sess.run(modelaccuracy, feed_dict={x: batch_x, y: batch_y})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
              "{:.6f}".format(mc) + ", Training Accuracy= " + \
              "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")


In [ ]:
#Speed accuracy

step = 1
totalMAE = 0
correctDirectionCount = 0
totalCount = 0
threshold = 12
# Calculate accuracy for 128 mnist test images
while step * batch_size < (len(images)-trainingIndex):
    test_data, test_label = nextTesting(batch_size)
    test_data = test_data.reshape((batch_size, n_input))
    test_label = test_label.reshape((batch_size, 8))
    predAccuracy = sess.run(modelaccuracy, feed_dict={x: test_data, y: test_label})
    totalMAE = totalMAE + predAccuracy
    results = sess.run(pred, feed_dict={x: test_data})
    print("Testing Accuracy:", predAccuracy)
    print(str(totalMAE/(step)))
    step+=1
